### The goal of this notebook is to demonstrate the following:
- programmatic query of calibration files for any of the active instruments
- using the results, chunk all of the darks into sets of darks covering a ~30 day period
    - these chunks of darks will then be combined to reject cosmic rays
    - For the time being I will **ignore** anneal boundaries. All cosmic rays affecting only a few pixels are rejected and so any hot pixels that are cured will be ignored across the anneal boundary will be removed
#### Author: Nate Miles

In [1]:
from astroquery.mast import Observations
from collections import defaultdict
from astropy.time import Time
import pandas as pd

- Originally, I was querying for every single dark every taken, then I was filtering that table by extension and date of observations.

- Instead, I should be filtering by date beforehand to get a much smaller table and then filter that table by extension to get only the RAW and SPT dataproducts

- In order to acheive this in a generalized manner without having to hard code dates, I will query for all darks, sort by date and then extract the first and last values to get the start and stop times

In [2]:
#Observations.enable_s3_hst_dataset()

In [9]:
obsTable = Observations.query_criteria(project='HST',
                                      instrument_name = 'WFPC2/WFC',
                                      obstype='cal',
                                      target_name = 'DARK')
df = obsTable.to_pandas()

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13375 entries, 0 to 13374
Data columns (total 31 columns):
dataproduct_type         13375 non-null object
calib_level              13375 non-null float64
obs_collection           13375 non-null object
obs_id                   13375 non-null object
target_name              13375 non-null object
s_ra                     13375 non-null float64
s_dec                    13375 non-null float64
t_min                    13375 non-null float64
t_max                    13375 non-null float64
t_exptime                13375 non-null float64
wavelength_region        13375 non-null object
filters                  13375 non-null object
em_min                   13375 non-null float64
em_max                   13375 non-null float64
target_classification    13373 non-null object
obs_title                12508 non-null object
t_obs_release            13375 non-null float64
instrument_name          13375 non-null object
proposal_pi              13375 non-n

In [11]:
df.head()


,dataproduct_type,calib_level,obs_collection,obs_id,target_name,s_ra,s_dec,t_min,t_max,t_exptime,...,project,s_region,jpegURL,dataURL,dataRights,mtFlag,srcDen,intentType,obsid,objID
0,image,2.0,HST,U43I9Q01R,DARK,152.769359,46.431595,50759.026540,50759.038110,1000.00000,...,HST,POLYGON GSC1 152.72574698 46.44535408 152.786...,mast:HST/product/u43i9q01r/u43i9q01r_c0f.jpg,mast:HST/product/u43i9q01r/u43i9q01r_c0f.fits,PUBLIC,False,NaN,calibration,2003633919,2012426589
1,image,2.0,HST,U68RA001R,DARK,354.430485,-32.320310,51845.192510,51845.204090,1000.00000,...,HST,POLYGON GSC1 354.45752280 -32.29642807 354.457...,mast:HST/product/u68ra001r/u68ra001r_c0f.jpg,mast:HST/product/u68ra001r/u68ra001r_c0f.fits,PUBLIC,False,NaN,calibration,2003682087,2012474757
2,image,2.0,HST,u4764f01r,DARK,215.798760,32.751048,50919.754326,50919.765901,1000.00224,...,HST,POLYGON GSC1 215.78896818 32.71903607 215.7630...,mast:HST/product/u4764f01r_c0f.jpg,mast:HST/product/u4764f01r_c0f.fits,PUBLIC,False,NaN,calibration,2003638127,2019710953
3,image,2.0,HST,u477dt01r,DARK,263.762599,-3.215614,51256.031403,51256.042977,999.99360,...,HST,POLYGON GSC1 263.72949299 -3.21577725 263.7612...,mast:HST/product/u477dt01r_c0f.jpg,mast:HST/product/u477dt01r_c0f.fits,PUBLIC,False,NaN,calibration,2003639085,2019723173
4,image,2.0,HST,u477g701r,DARK,218.673699,-39.638026,51285.014042,51285.025617,1000.00224,...,HST,POLYGON GSC1 218.65342407 -39.60889342 218.708...,mast:HST/product/u477g701r_c0f.jpg,mast:HST/product/u477g701r_c0f.fits,PUBLIC,False,NaN,calibration,2003639171,2019724544


In [12]:
start = Time(min(obsTable['t_min']), format='mjd')
stop = Time(max(obsTable['t_min']), format='mjd')
print(start.iso, stop.iso)
# very roundabout way of generating a list of MJD dates separated by a month
dates = pd.date_range(start=start.iso, end=stop.iso, freq='1MS')

dates = [Time(date.date().isoformat(),format='iso') for date in dates]

1993-12-17 14:57:17.378 2009-05-11 06:51:12.531


In [13]:
dates

[<Time object: scale='utc' format='iso' value=1994-01-01 00:00:00.000>,
 <Time object: scale='utc' format='iso' value=1994-02-01 00:00:00.000>,
 <Time object: scale='utc' format='iso' value=1994-03-01 00:00:00.000>,
 <Time object: scale='utc' format='iso' value=1994-04-01 00:00:00.000>,
 <Time object: scale='utc' format='iso' value=1994-05-01 00:00:00.000>,
 <Time object: scale='utc' format='iso' value=1994-06-01 00:00:00.000>,
 <Time object: scale='utc' format='iso' value=1994-07-01 00:00:00.000>,
 <Time object: scale='utc' format='iso' value=1994-08-01 00:00:00.000>,
 <Time object: scale='utc' format='iso' value=1994-09-01 00:00:00.000>,
 <Time object: scale='utc' format='iso' value=1994-10-01 00:00:00.000>,
 <Time object: scale='utc' format='iso' value=1994-11-01 00:00:00.000>,
 <Time object: scale='utc' format='iso' value=1994-12-01 00:00:00.000>,
 <Time object: scale='utc' format='iso' value=1995-01-01 00:00:00.000>,
 <Time object: scale='utc' format='iso' value=1995-02-01 00:00:0

In [14]:
date_ranges = zip(dates[::2], dates[1::2])

In [18]:
dates = next(date_ranges)

In [41]:
obsTable = Observations.query_criteria(project='HST',
                                      instrument_name='WFPC2/WFC',
                                      obstype ='cal',
                                      target_name='DARK',
                                      t_min=[dates[0].mjd, dates[1].mjd],
                                      t_exptime=[800, 2000])

In [42]:
products = Observations.get_product_list(obsTable)

In [43]:
products

obsID,obs_collection,dataproduct_type,obs_id,description,type,dataURI,productType,productGroupDescription,productSubGroupDescription,productDocumentationURL,project,prvversion,proposal_id,productFilename,size,parent_obsid
str10,str3,str5,str9,str67,str1,str40,str9,str28,str3,str1,str8,str19,str4,str23,int64,str10
2003586974,HST,image,u28u0b01t,DADS CGR file - Calibrated data WFPC/FOC,S,mast:HST/product/u28u0b01t_cgr.fits,AUXILIARY,--,CGR,--,CALWFPC2,"2.5.3 (Sep 4, 2008)",5562,u28u0b01t_cgr.fits,31680,2003586974
2003586974,HST,image,u28u0b01t,DADS DGR file - Raw data WFPC/FOC,S,mast:HST/product/u28u0b01t_dgr.fits,AUXILIARY,--,DGR,--,CALWFPC2,--,5562,u28u0b01t_dgr.fits,31680,2003586974
2003586974,HST,image,u28u0b01t,DADS JIF file,S,mast:HST/product/u28u0b01j_jif.fits,AUXILIARY,--,JIF,--,CALWFPC2,--,5562,u28u0b01j_jif.fits,17280,2003586974
2003586974,HST,image,u28u0b01t,DADS JIT file,S,mast:HST/product/u28u0b01j_jit.fits,AUXILIARY,--,JIT,--,CALWFPC2,--,5562,u28u0b01j_jit.fits,11520,2003586974
2003586974,HST,image,u28u0b01t,DADS Q0F file - Raw data quality WFPC/WFPC2/FOC/FOS/GHRS/HSP,S,mast:HST/product/u28u0b01t_q0f.fits,AUXILIARY,--,Q0F,--,CALWFPC2,--,5562,u28u0b01t_q0f.fits,5184000,2003586974
2003586974,HST,image,u28u0b01t,DADS Q0M file - Raw data quality WFPC2,S,mast:HST/product/u28u0b01t_q0m.fits,AUXILIARY,--,Q0M,--,CALWFPC2,--,5562,u28u0b01t_q0m.fits,5178240,2003586974
2003586974,HST,image,u28u0b01t,DADS Q1F file - Calibrated data quality WFPC/WFPC2/FOC/FOS/GHRS/HSP,S,mast:HST/product/u28u0b01t_q1f.fits,AUXILIARY,--,Q1F,--,CALWFPC2,--,5562,u28u0b01t_q1f.fits,103680,2003586974
2003586974,HST,image,u28u0b01t,DADS Q1M file - Calibrated data quality WFPC2,S,mast:HST/product/u28u0b01t_q1m.fits,AUXILIARY,--,Q1M,--,CALWFPC2,--,5562,u28u0b01t_q1m.fits,109440,2003586974
2003586974,HST,image,u28u0b01t,DADS SHM file - Engineering telemetry WFPC2,S,mast:HST/product/u28u0b01t_shm.fits,AUXILIARY,--,SHM,--,CALWFPC2,--,5562,u28u0b01t_shm.fits,34560,2003586974


In [44]:
products.columns

<TableColumns names=('obsID','obs_collection','dataproduct_type','obs_id','description','type','dataURI','productType','productGroupDescription','productSubGroupDescription','productDocumentationURL','project','prvversion','proposal_id','productFilename','size','parent_obsid')>

In [49]:
filtered_products = Observations.filter_products(products,
                                                 mrp_only=False,
                                                 productSubGroupDescription=['D0M','D1M'])

In [50]:
filtered_products


obsID,obs_collection,dataproduct_type,obs_id,description,type,dataURI,productType,productGroupDescription,productSubGroupDescription,productDocumentationURL,project,prvversion,proposal_id,productFilename,size,parent_obsid
str10,str3,str5,str9,str67,str1,str40,str9,str28,str3,str1,str8,str19,str4,str23,int64,str10
2003586974,HST,image,u28u0b01t,DADS D0M file - Raw exposure WFPC2,S,mast:HST/product/u28u0b01t_d0m.fits,SCIENCE,--,D0M,--,CALWFPC2,--,5562,u28u0b01t_d0m.fits,5178240,2003586974
2003586975,HST,image,u28u0c01t,DADS D0M file - Raw exposure WFPC2,S,mast:HST/product/u28u0c01t_d0m.fits,SCIENCE,--,D0M,--,CALWFPC2,--,5562,u28u0c01t_d0m.fits,5178240,2003586975
2003586976,HST,image,u28u0d01t,DADS D0M file - Raw exposure WFPC2,S,mast:HST/product/u28u0d01t_d0m.fits,SCIENCE,--,D0M,--,CALWFPC2,--,5562,u28u0d01t_d0m.fits,5178240,2003586976
2003586977,HST,image,u28u0e01t,DADS D0M file - Raw exposure WFPC2,S,mast:HST/product/u28u0e01t_d0m.fits,SCIENCE,--,D0M,--,CALWFPC2,--,5562,u28u0e01t_d0m.fits,5178240,2003586977
2003586978,HST,image,u28u0f01t,DADS D0M file - Raw exposure WFPC2,S,mast:HST/product/u28u0f01t_d0m.fits,SCIENCE,--,D0M,--,CALWFPC2,--,5562,u28u0f01t_d0m.fits,5178240,2003586978
2003586979,HST,image,u28u0g01t,DADS D0M file - Raw exposure WFPC2,S,mast:HST/product/u28u0g01t_d0m.fits,SCIENCE,--,D0M,--,CALWFPC2,--,5562,u28u0g01t_d0m.fits,5178240,2003586979
2003586980,HST,image,u28u0h01t,DADS D0M file - Raw exposure WFPC2,S,mast:HST/product/u28u0h01t_d0m.fits,SCIENCE,--,D0M,--,CALWFPC2,--,5562,u28u0h01t_d0m.fits,5178240,2003586980
2003586981,HST,image,u28u0i01t,DADS D0M file - Raw exposure WFPC2,S,mast:HST/product/u28u0i01t_d0m.fits,SCIENCE,--,D0M,--,CALWFPC2,--,5562,u28u0i01t_d0m.fits,5178240,2003586981
2003586982,HST,image,u28u0j01t,DADS D0M file - Raw exposure WFPC2,S,mast:HST/product/u28u0j01t_d0m.fits,SCIENCE,--,D0M,--,CALWFPC2,--,5562,u28u0j01t_d0m.fits,5178240,2003586982


In [51]:
filtered_products.columns

<TableColumns names=('obsID','obs_collection','dataproduct_type','obs_id','description','type','dataURI','productType','productGroupDescription','productSubGroupDescription','productDocumentationURL','project','prvversion','proposal_id','productFilename','size','parent_obsid')>

In [52]:
download_list = filtered_products['obsID'].tolist()

In [53]:
download_list

['2003586974',
 '2003586975',
 '2003586976',
 '2003586977',
 '2003586978',
 '2003586979',
 '2003586980',
 '2003586981',
 '2003586982',
 '2003586983',
 '2003586984',
 '2003586985',
 '2003586986',
 '2003586987',
 '2003586988',
 '2003586989',
 '2003586990',
 '2003586991',
 '2003586992',
 '2003586993',
 '2003586994',
 '2003586995',
 '2003586996',
 '2003586997',
 '2003586998']

In [55]:
manifest = Observations.download_products(download_list[:10],
                                          download_dir='./../crrejtab/WFPC2/',
                                          mrp_only=False,
                                         productSubGroupDescription=['D0M','D1M'])